In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("CW") \
    .getOrCreate()

sc = spark.sparkContext

In [2]:
sc

<SparkContext master=local[*] appName=CW>

In [3]:
from time import time
from preprocessing.Preprocessor import Preprocessor
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from metrics.metrics import Metrics

In [4]:
pp = Preprocessor(spark)

In [5]:
train = pp.to_bow('Cleaned Data/train.csv', min_frequency=275., remove_stop_words = False)
#train = train.sample(False, 0.9)
test =  pp.to_bow('Cleaned Data/test.csv')

In [6]:
"""lst_smooth = [0, 1.0, 100.0, 500, 1000]
lst_model = []
lst_model_transform = []


for i in lst_smooth:
    nb = NaiveBayes(smoothing=i, modelType="multinomial")
    model = nb.fit(train)
    lst_model.append(model)
    train_preds_and_labels = model.transform(train).select('prediction', 'label')
    lst_model_transform.append(train_preds_and_labels)"""

'lst_smooth = [0, 1.0, 100.0, 500, 1000]\nlst_model = []\nlst_model_transform = []\n\n\nfor i in lst_smooth:\n    nb = NaiveBayes(smoothing=i, modelType="multinomial")\n    model = nb.fit(train)\n    lst_model.append(model)\n    train_preds_and_labels = model.transform(train).select(\'prediction\', \'label\')\n    lst_model_transform.append(train_preds_and_labels)'

In [7]:
nb = NaiveBayes(smoothing = 1.0, modelType="multinomial")

In [8]:
%%time
model = nb.fit(train)

CPU times: user 9.82 ms, sys: 3.35 ms, total: 13.2 ms
Wall time: 6.81 s


In [9]:
"""model_predictions = []

for i in lst_model:
    predictions = i.transform(val)
    model_predictions.append(predictions)"""
predictions = model.transform(test)

In [10]:
test_m = Metrics()
test_fitting_check = model.transform(test).select('prediction', 'label')
test_m.f1(test_fitting_check)
test_m.accuracy(test_fitting_check)
test_m.confusion_matrix(test_fitting_check)
test_m.performance(test_fitting_check)

F1 score for given DataFrame: 72.1

Accuracy on given DataFrame: 71.9%

                Predictions
            negative    neutral     positive    
negative    2552        415         1164        
neutral     407         356         716         
positive    1008        900         8875        
(True labels are on the side)

Class     | Precision | Recall    | F1-score
negative    64.3        61.8        63.0        
neutral     21.3        24.1        22.6        
positive    82.5        82.3        82.4        



In [11]:
"""train_m = Metrics()
train_m.f1(model_predictions[0])
train_m.f1(model_predictions[1])
train_m.f1(model_predictions[2])
train_m.f1(model_predictions[3])
train_m.f1(model_predictions[4])"""

'train_m = Metrics()\ntrain_m.f1(model_predictions[0])\ntrain_m.f1(model_predictions[1])\ntrain_m.f1(model_predictions[2])\ntrain_m.f1(model_predictions[3])\ntrain_m.f1(model_predictions[4])'